In [16]:
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('TkAgg')
#%matplotlib inline
import BPV
import pattern_manipulation as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import rcParams
import pickle
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math

matplotlib.style.use('ggplot')
rcParams['text.latex.unicode'] = True
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsthm}', r'\usepackage{amsmath}', r'\usepackage{amssymb}', \
                                   r'\usepackage{amsfonts}'
#rcParams['text.latex.preamble']
#rcParams['text.latex.preamble'] = r'\usepackage{amsthm}'

data = BPV.Data()
data.read_csv("p.HEP.csv",False,False)
data.df.sort_index(by="p",inplace=True,ascending=True)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)
data.df

Wmin = 0.001
Wmax = 0.25
Wnsteps = 10
Wstep = (Wmax - Wmin)/Wnsteps
Wrange = np.arange(Wmin,Wmax,Wstep)

totpatterns = len(data.df)
Nmin = int(totpatterns/100)
Nmax = int(totpatterns/10)
Nnsteps = 10
Nstep = (Nmax - Nmin)/Nnsteps
Nrange = np.arange(Nmin,Nmax,Nstep, dtype='int')

nproblems = Wnsteps*Nnsteps


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
#CALCULATE EXACT SOLUTIONS

entropies = np.zeros((Wnsteps, Nnsteps))
rates = np.zeros((Wnsteps, Nnsteps))
cardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
indexes = {}

it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, nproblems, W, N,i,j))
    
    prbl_exact = BPV.BPV("pulp",data,N,W,time_solver=False)
    prbl_exact.solve()
    prbl_exact.pprint_solution()
    
    entropies[i,j] = prbl_exact.solution_entropy
    rates[i,j] = prbl_exact.solution_rate
    cardinalities[i,j] = prbl_exact.solution_cardinality
    indexes[(i,j)] = prbl_exact.solution.index
    it += 1
    



Solving problem 1 out of 100, for W = 0.001 and N = 209 i = 0  j = 0

Solver =  pulp 
Entropy =  0.012249985520693018 
Cardinality =  209 
Rate =  0.001


Solving problem 2 out of 100, for W = 0.001 and N = 397 i = 0  j = 1

Solver =  pulp 
Entropy =  0.0128895547810698 
Cardinality =  397 
Rate =  0.001


Solving problem 3 out of 100, for W = 0.001 and N = 585 i = 0  j = 2

Solver =  pulp 
Entropy =  0.013272531503314878 
Cardinality =  585 
Rate =  0.001


Solving problem 4 out of 100, for W = 0.001 and N = 773 i = 0  j = 3

Solver =  pulp 
Entropy =  0.013534286936765288 
Cardinality =  773 
Rate =  0.00099968


Solving problem 5 out of 100, for W = 0.001 and N = 961 i = 0  j = 4

Solver =  pulp 
Entropy =  0.013718533011450812 
Cardinality =  961 
Rate =  0.00099936


Solving problem 6 out of 100, for W = 0.001 and N = 1149 i = 0  j = 5

Solver =  pulp 
Entropy =  0.01384596789348725 
Cardinality =  1149 
Rate =  0.00099936


Solving problem 7 out of 100, for W = 0.001 and N = 13

KeyboardInterrupt: 

In [7]:
zeroindexes = (entropies == 0).nonzero()
totzeros = len(zeroindexes[0])
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
for i,j in zeroindexes:
    print(i,j,totzeros)

1 4 3
1 5 3
1 8 3


In [ ]:
#RECALCULATE SOLUTIONS NOT SOLVED BY PULP WITH GLPK
zeroindexes = (entropies == 0).nonzero()
totzeros = len(zeroindexes[0])
zeroindexes = zip(zeroindexes[0], zeroindexes[1])
it = 0
for i,j in zeroindexes:
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, totzeros, Wrange[i], Nrange[j],i,j))
    glpsol = BPV.BPV("glpk",data,Nrange[j],Wrange[i],time_solver=True)
    glpsol.solve()
    entropies[i,j] = glpsol.solution_entropy
    rates[i,j] = glpsol.solution_rate
    cardinalities[i,j] = glpsol.solution_cardinality
    indexes[(i,j)] = glpsol.solution.index
    it+=1



Solving problem 0 out of 23, for W = 0.0093 and N = 10 i = 1  j = 0


Solving problem 1 out of 23, for W = 0.017599999999999998 and N = 70 i = 2  j = 15

In [ ]:
#RECALCULATE SOLUTIONS NOT SOLVED BY PULP WITH GLPK, reversing indexes
zeroindexes = (entropies == 0).nonzero()
totzeros = len(zeroindexes[0])
zeroindexes = zip(zeroindexes[0], zeroindexes[1])

data.df.sort_index(by="p",inplace=True,ascending=False)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)

def reverseindex(pdindex,totpatterns=512):
    newindex = []
    for i in pdindex:
        newindex.append(totpatterns -1 -i)
    return(pd.Index(newindex))
  
#function test
#a = [0,1,2,3]
#print(a,reverseindex(a))

it = 0
for i,j in zeroindexes:
    print("\n\nSolving problem {0} out of {1}, for W = {2} and N = {3} i = {4}  j = {5}".format(it, totzeros, Wrange[i], Nrange[j],i,j))
    glpsol = BPV.BPV("glpk",data,Nrange[j],Wrange[i],time_solver=True)
    glpsol.solve()
    entropies[i,j] = glpsol.solution_entropy
    rates[i,j] = glpsol.solution_rate
    cardinalities[i,j] = glpsol.solution_cardinality
    indexes[(i,j)] = glpsol.solution.index
    it+=1

    
data.df.sort_index(by="p",inplace=True,ascending=True)
data.df.set_index(pd.Index([j for j in range(len(data.df))]), inplace=True)
data.df



Solving problem 0 out of 23, for W = 0.0093 and N = 10 i = 1  j = 0


Solving problem 1 out of 23, for W = 0.017599999999999998 and N = 70 i = 2  j = 15

In [49]:
#SAVE EXACT SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot11.entropies",r'wb')
f2 = open("/home/renato/tesi/code/data/plot11.rates",r'wb')
f3 = open("/home/renato/tesi/code/data/plot11.cardinalities",r'wb')
f4= open("/home/renato/tesi/code/data/plot11.indexes",r'wb')

pickle.dump(entropies,f1)
pickle.dump(rates,f2)
pickle.dump(cardinalities,f3)
pickle.dump(indexes,f4)


In [2]:
#LOAD EXACT SOLUTIONS
f1 = open("/home/renato/tesi/code/data/plot11.entropies",r'rb')
f2 = open("/home/renato/tesi/code/data/plot11.rates",r'rb')
f3 = open("/home/renato/tesi/code/data/plot11.cardinalities",r'rb')
f4 = open("/home/renato/tesi/code/data/plot11.indexes",r'rb')

entropies = pickle.load(f1)
rates = pickle.load(f2)
cardinalities = pickle.load(f3)
indexes = pickle.load(f4)

In [9]:
#REVERSE INDEXES, IF THEY WERE SAVED AS DECREASING FOR P 
def reverseindex(pdindex,totpatterns=512):
    newindex = []
    for i in pdindex:
        newindex.append(totpatterns -1 -i)
    return(pd.Index(newindex))
print(indexes[0,0])
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    indexes[i,j] = reverseindex(indexes[i,j])
print(indexes[0,0])

Int64Index([257, 258, 262, 263, 265, 266, 267, 268, 269, 270], dtype='int64')
Int64Index([254, 253, 249, 248, 246, 245, 244, 243, 242, 241], dtype='int64')


In [9]:
#CALCULATE HEURISTIC SOLUTIONS
heur_entropies = np.zeros((Wnsteps, Nnsteps))
heur_rates = np.zeros((Wnsteps, Nnsteps))
heur_cardinalities = np.zeros((Wnsteps, Nnsteps), dtype='int')
heur_indexes = {}

it = 1
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    W = Wrange[i]
    N = Nrange[j]
    print("Solving problem {0} out of {1}, for W = {2} and N = {3}".format(it, nproblems, W, N))
    
    prbl_heur = BPV.BPV("heuristic",data,N,W,time_solver=False)
    prbl_heur.solve()
    
    heur_entropies[i,j] = prbl_heur.solution_entropy
    heur_rates[i,j] = prbl_heur.solution_rate
    heur_cardinalities[i,j] = prbl_heur.solution_cardinality
    heur_indexes[(i,j)] = prbl_heur.solution.index
    it += 1

Solving problem 1 out of 100, for W = 0.001 and N = 10
Solving problem 2 out of 100, for W = 0.001 and N = 24
Solving problem 3 out of 100, for W = 0.001 and N = 38
Solving problem 4 out of 100, for W = 0.001 and N = 52
Solving problem 5 out of 100, for W = 0.001 and N = 66
Solving problem 6 out of 100, for W = 0.001 and N = 80
Solving problem 7 out of 100, for W = 0.001 and N = 94
Solving problem 8 out of 100, for W = 0.001 and N = 108
Solving problem 9 out of 100, for W = 0.001 and N = 122
Solving problem 10 out of 100, for W = 0.001 and N = 136
Solving problem 11 out of 100, for W = 0.0259 and N = 10
Solving problem 12 out of 100, for W = 0.0259 and N = 24
Solving problem 13 out of 100, for W = 0.0259 and N = 38
Solving problem 14 out of 100, for W = 0.0259 and N = 52
Solving problem 15 out of 100, for W = 0.0259 and N = 66
Solving problem 16 out of 100, for W = 0.0259 and N = 80
Solving problem 17 out of 100, for W = 0.0259 and N = 94
Solving problem 18 out of 100, for W = 0.0259 a

In [48]:
#SAVE HEURISTIC SOLUTIONS
heur_f1 = open("/home/renato/tesi/code/data/plot11.heur_entropies",r'wb')
heur_f2 = open("/home/renato/tesi/code/data/plot11.heur_rates",r'wb')
heur_f3 = open("/home/renato/tesi/code/data/plot11.heur_cardinalities",r'wb')
heur_f4= open("/home/renato/tesi/code/data/plot11.heur_indexes",r'wb')

pickle.dump(heur_entropies,heur_f1)
pickle.dump(heur_rates,heur_f2)
pickle.dump(heur_cardinalities,heur_f3)
pickle.dump(heur_indexes,heur_f4)


In [12]:
#LOAD HEURISTIC SOLUTIONS
heur_f1 = open("/home/renato/tesi/code/data/plot11.heur_entropies",r'rb')
heur_f2 = open("/home/renato/tesi/code/data/plot11.heur_rates",r'rb')
heur_f3 = open("/home/renato/tesi/code/data/plot11.heur_cardinalities",r'rb')
heur_f4 = open("/home/renato/tesi/code/data/plot11.heur_indexes",r'rb')

heur_entropies = pickle.load(heur_f1)
heur_rates = pickle.load(heur_f2)
heur_cardinalities = pickle.load(heur_f3)
heur_indexes = pickle.load(heur_f4)

In [13]:
#REVERSE INDEXES, IF THEY WERE SAVED AS DECREASING FOR P 
def reverseindex(pdindex,totpatterns=512):
    newindex = []
    for i in pdindex:
        newindex.append(totpatterns -1 -i)
    return(pd.Index(newindex))
print(heur_indexes[0,0])
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    heur_indexes[i,j] = reverseindex(heur_indexes[i,j])
print(heur_indexes[0,0])

Int64Index([265, 264, 263, 266, 262, 261, 260, 259, 258], dtype='int64')
Int64Index([246, 247, 248, 245, 249, 250, 251, 252, 253], dtype='int64')


In [8]:
# EXACT AND HEUR MEGA PLOT A
fig = plt.figure(figsize=(9,25))
#ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
elev = 22
azimuth = -135
#elev = 40
#azimuth = 47
al = 1
cb_shrink = 0.1
cb_aspect = 10
label_size = 16
title_size = 20
title_rotation = 'horizontal'
title_x = 0.4
title_y = 0
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colormap = cm.gnuplot
_dpi = 80

#gs = gridspec.GridSpec(2,7, width_ratios=[1,0.1,10,0.1,10,0.1,10], height_ratios=[1,1])

gs = gridspec.GridSpec(4,2, width_ratios=[1,100], height_ratios=[1,10,10,10])

title1 = plt.subplot(gs[0,:2])
cex_entropy = plt.subplot(gs[1,0])
ex_entropy = plt.subplot(gs[1,1], projection='3d') #exact entropy

cex_rate = plt.subplot(gs[2,0])
ex_rate = plt.subplot(gs[2,1], projection='3d') #exact rate

cex_card = plt.subplot(gs[3,0])
ex_card = plt.subplot(gs[3,1], projection='3d') #exact cardinality

title1.axis('off')
title1.text(title_x,title_y,"Exact Solution",fontsize=title_size, rotation=title_rotation)

surf_e = ex_entropy.plot_surface(X, Y, entropies.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_entropy.set_xlabel('W',fontsize=label_size)
ex_entropy.set_ylabel('N',fontsize=label_size)
ex_entropy.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_entropy.set_zlabel('Entropy',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_entropy.view_init(elev,azimuth)
plt.colorbar(surf_e, cax=cex_entropy)



surf_r = ex_rate.plot_surface(X, Y, rates.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_rate.set_xlabel('W',fontsize=label_size)
ex_rate.set_ylabel('N',fontsize=label_size)
ex_rate.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_rate.set_zlabel('Rate',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ex_rate.view_init(elev,azimuth)
plt.colorbar(surf_r, cax=cex_rate)

surf_c = ex_card.plot_surface(X, Y, cardinalities.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
ex_card.set_xlabel('W',fontsize=label_size)
ex_card.set_ylabel('N',fontsize=label_size)
ex_card.zaxis.set_rotate_label(False)  # disable automatic rotation
ex_card.set_zlabel('Cardinality',rotation='vertical',fontsize=label_size, horizontalalignment='right',)
ex_card.view_init(elev,azimuth)
plt.colorbar(surf_c, cax=cex_card)

fig.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/NWsummaryExact.pdf",  bbox_inches='tight')
plt.show()

In [10]:
# EXACT AND HEUR MEGA PLOT  B
fig = plt.figure(figsize=(9,25))
#ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
elev = 22
azimuth = -135
#elev = 40
#azimuth = 47
al = 1
cb_shrink = 0.1
cb_aspect = 10
label_size = 16
title_size = 20
title_rotation = 'horizontal'
title_x = 0.4
title_y = 0
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
colormap = cm.gnuplot
_dpi = 80

#gs = gridspec.GridSpec(2,7, width_ratios=[1,0.1,10,0.1,10,0.1,10], height_ratios=[1,1])

gs = gridspec.GridSpec(4,2, width_ratios=[1,100], height_ratios=[1,10,10,10])

title2 = plt.subplot(gs[0,:])
cheur_entropy = plt.subplot(gs[1,0])
heur_entropy = plt.subplot(gs[1,1], projection='3d') #heur entropy

cheur_rate = plt.subplot(gs[2,0])
heur_rate = plt.subplot(gs[2,1], projection='3d') #heur rate

cheur_card = plt.subplot(gs[3,0])
heur_card = plt.subplot(gs[3,1], projection='3d') #heur cardinality




title2.axis('off')
title2.text(title_x,title_y,"Heuristic Solution",fontsize=title_size, rotation=title_rotation)

surf_he = heur_entropy.plot_surface(X, Y, heur_entropies.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
heur_entropy.set_xlabel('W',fontsize=label_size)
heur_entropy.set_ylabel('N',fontsize=label_size)
heur_entropy.zaxis.set_rotate_label(False)  # disable automatic rotation
heur_entropy.set_zlabel('Entropy',rotation='vertical',fontsize=label_size, horizontalalignment='right')
heur_entropy.view_init(elev,azimuth)
plt.colorbar(surf_he, cax=cheur_entropy)



surf_hr = heur_rate.plot_surface(X, Y, heur_rates.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
heur_rate.set_xlabel('W',fontsize=label_size)
heur_rate.set_ylabel('N',fontsize=label_size)
heur_rate.zaxis.set_rotate_label(False)  # disable automatic rotation
heur_rate.set_zlabel('Rate',rotation='vertical',fontsize=label_size, horizontalalignment='right')
heur_rate.view_init(elev,azimuth)
plt.colorbar(surf_hr, cax=cheur_rate)


surf_hc = heur_card.plot_surface(X, Y, heur_cardinalities.T, rstride=1, cstride=1, alpha=al, cmap=colormap)
heur_card.set_xlabel('W',fontsize=label_size)
heur_card.set_ylabel('N',fontsize=label_size)
heur_card.zaxis.set_rotate_label(False)  # disable automatic rotation
heur_card.set_zlabel('Cardinality',rotation='vertical',fontsize=label_size, horizontalalignment='right')
heur_card.view_init(elev,azimuth)
plt.colorbar(surf_hc, cax=cheur_card)

fig.tight_layout()
#plt.savefig("/home/renato/tesi/testogit/img/NWsummaryHeur.pdf",  bbox_inches='tight')
plt.show()

In [11]:
#CALCULATE GRADIENT, NOT INTERVAL MEASURE, RELATIVE ERROR
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(Wrange, Nrange)
gx, gy = np.gradient(entropies)
grad = np.zeros((Wnsteps, Nnsteps))
not_interval_measure =  np.zeros((Wnsteps, Nnsteps))
relerr = np.zeros((Wnsteps,Nnsteps))
for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    grad[i,j] = np.sqrt(gx[i,j]**2 + gy[i,j]**2)
    not_interval_measure[i,j] = BPV.not_interval_measure(indexes[i,j])
    relerr[i,j] = (entropies[i,j] - heur_entropies[i,j])/entropies[i,j]

In [12]:
#RELATIVE ERRROR
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.6
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
#gradoffset = 5*relerr.min()
gradoffset = 0
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

relmin = relerr.min()
relmax = relerr.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, relerr.T, rstride=1, cstride=1, alpha=al, \
                         vmin=relmin, vmax=relmax/2, cmap=colormap)
cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('Relative Error',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWrelerr.pdf",  bbox_inches='tight')
plt.show()

In [13]:
#NOT INTERVAL MEASURE
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(10,5))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 35
azimuth = 31
al = 0.65
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 5*relerr.min()
gradlinew = 3
gradcolor = 'k'
ncountourlines = 7
ext3d = False
colormap = cm.gnuplot

mingr = grad.min()
maxgr = grad.max()
grcontourlevels = list(np.arange(mingr,maxgr,(maxgr-mingr)/ncountourlines ))

nimin = not_interval_measure.min()
nimax = not_interval_measure.max()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf_r = ax.plot_surface(X, Y, not_interval_measure.T, rstride=1, cstride=1, alpha=al,\
                         vmin=nimin,vmax=nimax, cmap=colormap)
cset = ax.contour(X, Y, grad.T, zdir='z', offset=gradoffset, colors=gradcolor, \
                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$\mathcal{I}(S)$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWnimeasure.pdf",  bbox_inches='tight')
plt.show()

In [14]:
#MEAN VALUES OF INDEXES 1
heur_solmean = np.zeros((Wnsteps,Nnsteps))
ex_solmean = np.zeros((Wnsteps,Nnsteps))

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    hm = (np.array(heur_indexes[i,j])).mean()
    if math.isnan(hm):
        print(i,j, heur_indexes[i,j])
    heur_solmean[i,j] = hm
    ex_solmean[i,j] = (np.array(indexes[i,j])).mean()
    
fig = plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

hmin = 1000
hmax = -2000
for coords,d in np.ndenumerate(heur_solmean):
    if math.isnan(d) == False:
        if d < hmin:
            hmin=d
        elif d> hmax:
            hmax=d
hmin *= 1
hmax *= 1

emin = 1000
emax = -2000
for coords,d in np.ndenumerate(ex_solmean):
    if math.isnan(d) == False:
        if d < emin:
            emin=d
        elif d> emax:
            emax=d
emin *= 1
emax *= 1

X, Y = np.meshgrid(Wrange, Nrange)
elev = 15
azimuth = 124
al = 0.7
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 0.2
ncountourlines = 10
colormap = cm.gnuplot

ax = fig.add_subplot(2, 1, 1, projection='3d')
surf_e = ax.plot_surface(X, Y, ex_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=emin,vmax=emax)
ax.set_title('Exact Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)


grcontourlevels = list(np.arange(hmin,hmax,(hmax-hmin)/ncountourlines ))

ax = fig.add_subplot(2, 1, 2, projection='3d')
surf_r = ax.plot_surface(X, Y, heur_solmean.T, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=hmin,vmax=hmax)
#cset = ax.contour(X, Y, heur_solmean.T, zdir='y', offset=gradoffset, colors=gradcolor, \
#                  levels=grcontourlevels, linewidths=gradlinew, extend3d=ext3d)
ax.set_title('Heuristic Solution')
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_r, shrink=cb_shrink, aspect=cb_aspect)

#plt.savefig("/home/renato/tesi/testogit/img/NWmediumindexes.pdf",  bbox_inches='tight')

plt.show()

/opt/anaconda/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [15]:
#DIFF OF MEAN INDEXES
heur_solmean = np.zeros((Wnsteps,Nnsteps))
ex_solmean = np.zeros((Wnsteps,Nnsteps))

for i,j in [(x,y) for x in range(Wnsteps) for y in range(Nnsteps)]:
    hm = (np.array(heur_indexes[i,j])).mean()
    if math.isnan(hm):
        print(i,j, heur_indexes[i,j])
    heur_solmean[i,j] = hm
    ex_solmean[i,j] = (np.array(indexes[i,j])).mean()
    
#fig = plt.figure(figsize=(10,10))
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

X, Y = np.meshgrid(Wrange, Nrange)
elev = 24
azimuth = -66
al = 1
cb_shrink = 0.5
cb_aspect = 10
label_size = 16
gradoffset = 0
gradlinew = 2
ncountourlines = 10
colormap = cm.gnuplot

fig = plt.figure()
ax = fig.gca(projection='3d')
#diff = ex_solmean.T - heur_solmean.T
diff = heur_solmean.T - ex_solmean.T
dmin = 10
dmax = -20
for coords,d in np.ndenumerate(diff):
    if math.isnan(d) == False:
        if d < dmin:
            dmin=d
        elif d> dmax:
            dmax=d

surf_e = ax.plot_surface(X, Y, diff, rstride=1, cstride=1, alpha=al, cmap=colormap,\
                        vmin=dmin, vmax=dmax)
ax.set_xlabel('W')
ax.set_ylabel('N')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$m(S_{\\text{heur}}) - m(S_{\\text{ex}})$',rotation='vertical',fontsize=label_size, horizontalalignment='right')
ax.view_init(elev,azimuth)
fig.colorbar(surf_e, shrink=cb_shrink, aspect=cb_aspect)
#plt.savefig("/home/renato/tesi/testo/img/NWdiffmediumindex.pdf",  bbox_inches='tight')
#plt.savefig("/home/renato/tesi/testogit/img/NWdiffmediumindex.pdf")

plt.show()

/opt/anaconda/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
